In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,ID,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,...,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND,Purchase
0,ID0002,37,1,2,2,8,1,4,1,4,...,0,0,0,1,0,0,0,0,0,0
1,ID0004,9,1,3,3,3,2,3,2,4,...,0,0,0,1,0,0,0,0,0,0
2,ID0005,40,1,4,2,10,1,4,1,4,...,0,0,0,1,0,0,0,0,0,0
3,ID0006,23,1,2,1,5,0,5,0,5,...,0,0,0,0,0,0,0,0,0,0
4,ID0009,33,1,2,4,8,0,1,3,6,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_new=df.iloc[:,1:6]

In [5]:
df_new['MBERHOOG']=df['MBERHOOG']

In [6]:
df_new.head()

,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MBERHOOG
0,37,1,2,2,8,0
1,9,1,3,3,3,4
2,40,1,4,2,10,0
3,23,1,2,1,5,2
4,33,1,2,4,8,1


In [27]:
X=df_new.iloc[:,:].values
y=df.iloc[:,86].values

In [25]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [0,3,4])
X = onehotencoder.fit_transform(X).toarray()

In [26]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

ValueError: Found input variables with inconsistent numbers of samples: [2553, 2618]

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X= StandardScaler()
X_train= sc_X.fit_transform(X_train)
X_test= sc_X.fit(X_test)

In [10]:
from xgboost import XGBClassifier
xg_classifier = XGBClassifier()
xg_classifier.fit(X_train, y_train)

C:\Users\Jaadugar\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [11]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = xg_classifier, X = X_train, y = y_train, cv = 10)
print(accuracies.mean())
print(accuracies.std())

0.937921775572
0.00200346043228


In [12]:
# Predicting the Test set results
y_pred_xg = xg_classifier.predict(X_test)

In [13]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_xg)
print(cm)

[[491   0]
 [ 33   0]]


In [14]:
df_test=pd.read_csv('test.csv')
df_new=df_test.iloc[:,1:6]
df_new['MBERHOOG']=df_test['MBERHOOG']
X_test=df_new.iloc[:,:].values

In [16]:
X_test = onehotencoder.fit_transform(X_test).toarray()

In [17]:
df_new.head()

,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MBERHOOG
0,33,1,3,2,8,1
1,37,1,2,2,8,0
2,39,2,3,2,9,0
3,33,1,2,3,8,2
4,33,1,2,3,8,1


In [18]:
# Predicting the Test set results
y_pred_xg = xg_classifier.predict(X_test)

In [19]:
#Saving the file
result=pd.read_csv('update.csv')
result['Purchase']=y_pred_xg

In [20]:
result.to_csv('New.csv',index=False)